# RQ1

## Todo
- Clean up punctuation and filler words from the set (eg. _is_, _was_, _at_, _on_)

In [42]:
import pandas as pd
import nltk as nt
from afinn import Afinn
# nt.download('punkt') # uncomment this every time you run this program!

# read the data
rdata = pd.read_csv('Datafiniti_Hotel_Reviews.csv', header=0, sep=',', on_bad_lines='skip')

# extract the title and body text of each review into a large list
rtext, rtitle = rdata['reviews.text'].astype(str), rdata['reviews.title'].astype(str)

# remove "... More" from reviews (if it exists)
# suubstitute every instance of "... More" with an empty string
rtext = rtext.str.replace('((Bad|Good):)|\.\.\. More', '', regex=True)

# tokenise
rtext_tk = rtext.apply(nt.word_tokenize)

# create a giant array for sentiments
rtext_stm = []

afn = Afinn() # initialise an afinn instance

# loop through the tokenised list and assign each word a score
for rtk in rtext_tk:
	sentiments = [afn.score(r) for r in rtk]
	# print(sentiments)
	rtext_stm.append(sentiments)

# testing
print(f"Word #1 in Rating #2 ('{rtext_tk[1][0]}') has a score of: {rtext_stm[1][0]}.")
print(f"Word #42 in Rating #69 ('{rtext_tk[68][41]}') has a score of: {rtext_stm[68][41]}")

Word #1 in Rating #2 ('Amazing') has a score of: 4.0.
Word #42 in Rating #69 ('fresh') has a score of: 1.0
